In [1]:
from PIL import Image, ImageDraw, ImageFont
import os,glob
import numpy as np
import random
import matplotlib.pylab as plt

import cv2


In [2]:
fontTypes=[cv2.FONT_HERSHEY_SIMPLEX, cv2.FONT_HERSHEY_PLAIN, cv2.FONT_HERSHEY_DUPLEX, cv2.FONT_HERSHEY_COMPLEX, cv2.FONT_HERSHEY_TRIPLEX, cv2.FONT_HERSHEY_COMPLEX_SMALL,cv2.FONT_ITALIC]
fontsize=[21,9,21,21,22,16,23]

import string
rus='абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
rus= u'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
digit='0123456789'
# charactersList = list(string.ascii_lowercase)+list(string.ascii_uppercase)+list(rus)+list(rus.upper())+list(digit)
charactersList = list(string.ascii_lowercase)+list(string.ascii_uppercase)+list(digit)


colorlist=[]


image = img = np.zeros((32,32,3), np.uint8)+255
h,w,_ = image.shape

for k in range(0,200,20):             
                 colorlist.append((k,k,k))

In [3]:
# colorlist=[(0,0,0)]
# colorlist=[0]

In [5]:
def vertical_enh(image0,k=2):
    new_height=int(image0.shape[0]*k)
    dsize = (image0.shape[1], new_height)
    output = cv2.resize(image0, dsize, interpolation = cv2.INTER_AREA)
    m=new_height-image0.shape[0]
    
    if m%2==0: 
        m=m//2
        output=output[m:new_height-m,:]
    else:
        m=m//2
        output=output[m:new_height-m-1,:]
        
#     plt.imshow(output)
    return output
    
def horizontal_enh(image0,k=2):
    new_width=int(image0.shape[1]*k)
    dsize = (new_width,image0.shape[0])
    output = cv2.resize(image0, dsize, interpolation = cv2.INTER_AREA)
    m=new_width-image0.shape[1]
    
    if m%2==0: 
        m=m//2
        output=output[:,m:new_width-m]
    else:
        m=m//2
        output=output[:,m:new_width-m-1]
        

#     print(output.shape,new_width)
#     plt.imshow(output)    
    return output
    
# vertical_enh(image0,k=1.2)
# horizontal_enh(image0,k=2.9)

In [6]:
dataX=[]
dataY=[]

In [18]:
klen=0    
for text in charactersList[:]:
    print(f'{text} symbol begins')
    for i in range(7):
        fontScale =32//fontsize[i]
#         fontScale =1
        
        org = (int(h/2)-fontsize[i]*fontScale//3, int(w/2)+fontsize[i]*fontScale*2//3)
        font = fontTypes[i]
    #     org = (int(h/2), int(w/2))
    #     fontScale = 10
        for color_num,color in enumerate(colorlist):  
            max_th=3
            if i==5: max_th=2
            if i==1: max_th=1
            for thickness in range(1,max_th+1):
#                 print('thicknes=',thickness)
                img = np.zeros((48,48), np.uint8)+255
                
                image0 = cv2.putText(img, text, org, font, fontScale, 
                             color, thickness, cv2.LINE_AA, False)
#                 print (image0.shape)

                name="letter_dataset\ "+str(text)+'_font'+str(i)+'_clr'+str(color_num)+'_th'+str(thickness)+".jpg"
#                 isWritten = cv2.imwrite(name, image0)
                dataX.append(image0)
                dataY.append(text)
                if dataX[-1].shape!=(48,48):
                    print(klen,1,dataX[klen].shape,image0.shape)
                    break
                klen+=1
         

                k = 1
                while k<1.6:
                    k=int(k*10+2)/10
                    vert = vertical_enh(image0,k)
                    name="letter_dataset\ "+str(text)+'_font'+str(i)+'_clr'+str(color_num)+'_th'+str(thickness)+'_vert'+str(k)+".jpg"
#                     isWritten = cv2.imwrite(name, vert)                    
                    dataY.append(text)
                    dataX.append(vert)
                    if dataX[-1].shape!=(48,48):
                        print(klen,2,dataX[-1].shape,dataX[klen].shape,vert.shape)
                        break
                    klen+=1
                    hor = horizontal_enh(image0,k)
                    name="letter_dataset\ "+str(text)+'_font'+str(i)+'_clr'+str(color_num)+'_th'+str(thickness)+'_hor'+str(k)+".jpg"
#                     isWritten = cv2.imwrite(name, hor)
                    dataX.append(hor)      
                    dataY.append(text)
                    if dataX[-1].shape!=(48,48):
                        print(klen,3)
                        break
                    klen+=1


import pickle as pkl
dataX=np.array(dataX)
dataY=np.array(dataY)

#to save it
with open("letters.pkl", "ab") as f:
    pkl.dump([dataX, dataY], f)

In [19]:
#to load it
with open("letters.pkl", "rb") as f:
    dataX1, dataY1 = pkl.load(f)